# Kubeflow pipeline generator
## Generate a pipeline with a single Nuclio function

In [18]:
#!pip install kfp
import sys,os
sys.path.append('../../../igxartifacts/lib')
sys.path.append('../../../pipelines/sdk/python')

In [19]:
import kfp
from kfp import dsl
from igz import v3io_cred, xcp_op, mount_v3io

## Change to your cluster's endpoint for pipelines

In [44]:
%env KF_PIPELINES_UI_ENDPOINT=https://dashboard.default-tenant.app.mdl0716.iguazio-cd2.com/pipelines

env: KF_PIPELINES_UI_ENDPOINT=https://dashboard.default-tenant.app.mdl0716.iguazio-cd2.com/pipelines


## Change to location of the location_based_recommendation folder
### or use a diffrent folder and different Jupyter notebook

### The jupyter notebook should generate a Nuclio function with all its specs and pre-requisities

The Jupyter notebok should set the environment variables for V3IO
```
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
```

In [47]:
NOTEBOOKS_DIR = os.getenv('NOTEBOOKS_DIR','/User/GitHub/tutorials/demos/location_based_recommendation')

## N_GEN is the notebook to run in Kubeflow step

In [47]:
N_GEN = os.getenv('N_GEN','02-nuclio-generator')
WORK_DIR = os.getenv('WORK_DIR','/v3io/bigdata/lbr')
LBR_PATH  = os.getenv('LBR_PATH','/v3io/bigdata/lbr')

In [48]:
print(NOTEBOOKS_DIR)

/User/GitHub/tutorials/demos/location_based_recommendation


In [49]:
%env NOTEBOOKS_DIR={NOTEBOOKS_DIR}
%env N_GEN={N_GEN}
%env LGBM_NB={LGBM_NB}

env: NOTEBOOKS_DIR=/User/GitHub/tutorials/demos/location_based_recommendation
env: N_GEN=02-nuclio-generator
env: LGBM_NB={LGBM_NB}


In [61]:
!cp {NOTEBOOKS_DIR}/{N_GEN}.ipynb {LBR_PATH}/.

In [62]:
nuclio_deploy = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/deploy/component.yaml')
nuclio_invoke = kfp.components.load_component(url='https://raw.githubusercontent.com/kubeflow/pipelines/master/components/nuclio/invoker/component.yaml')

In [39]:
##NOT IN USE
def run_step(NAME,CMD,LBR_PATH):
    """Print env vars."""
    cop = dsl.ContainerOp(
        name=NAME,
        image='python:3.6-jessie',
        command=['ls /lbr',CMD],
    )
    cop.apply(v3io_cred())
    cop.apply(mount_v3io(container='bigdata', sub_path=LBR_PATH, mount_path='/lbr'))
    return cop

In [63]:
@dsl.pipeline(
    name='SCRATCH pipeline',
    description='Runnign LBR in kubeflow'
)
def my_pipeline(
   txt='good morningf',
   val=7,
):
    build = nuclio_deploy(url='/lbr/02-nuclio-generator.ipynb', name='generator1', project='myproj',dashboard='http://default-tenant-nuclio-dashboard:8070').\
        apply(mount_v3io(container='bigdata', sub_path='/lbr', mount_path='/lbr')).apply(v3io_cred())


    

In [64]:
kfp.compiler.Compiler().compile(my_pipeline, 'scratchpipe.yaml')

In [65]:
EXPERIMENT_NAME = 'Scratch pipeline'

In [66]:
client = kfp.Client(namespace='default-tenant')
arguments = {'txt': 'good evening', 'val': 7}
experiment = client.create_experiment(EXPERIMENT_NAME)
run_name = my_pipeline.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, 'scratchpipe.yaml', arguments)